## 0. Understanding Data


In [2]:
# import required modules
import sqlite3
import pandas as pd

In [3]:
# create a connection 
conn = sqlite3.connect('star_wars.db')

In [51]:
# take a look at the stored tables
tables = ['people', 'films', 'starships', 'vehicles', 'species', 'planets']

for table in tables:
    display(table)
    display(pd.read_sql(f'SELECT * FROM {table} LIMIT 1', conn))

'people'

,name,birth_year,eye_color,gender,hair_color,height,mass,skin_color,homeworld,films,species,starships,vehicles,url,created,edited,unique_id
0,Luke Skywalker,19BBY,blue,male,blond,172,77,fair,1,"[""https://swapi.dev/api/films/1/"", ""https://sw...",[],"[""https://swapi.dev/api/starships/12/"", ""https...","[""https://swapi.dev/api/vehicles/14/"", ""https:...",https://swapi.dev/api/people/1/,2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,1


'films'

,title,episode_id,opening_crawl,director,producer,release_date,species,starships,vehicles,characters,planets,url,created,edited,unique_id
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[""https://swapi.dev/api/species/1/"", ""https://...","[""https://swapi.dev/api/starships/2/"", ""https:...","[""https://swapi.dev/api/vehicles/4/"", ""https:/...","[""https://swapi.dev/api/people/1/"", ""https://s...","[""https://swapi.dev/api/planets/1/"", ""https://...",https://swapi.dev/api/films/1/,2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,1


'starships'

...

,name,model,vehicle_class,manufacturer,length,cost_in_credits,crew,passengers,max_atmosphering_speed,cargo_capacity,consumables,films,pilots,url,created,edited,unique_id
0,Sand Crawler,Digger Crawler,wheeled,Corellia Mining Corporation,36.8,150000,46,30,30,50000,2 months,"[""https://swapi.dev/api/films/1/"", ""https://sw...",[],https://swapi.dev/api/vehicles/4/,2014-12-10T15:36:25.724000Z,2014-12-20T21:30:21.661000Z,4


'species'

,name,average_height,average_lifespan,classification,designation,eye_colors,hair_colors,homeworld,language,skin_colors,people,films,url,created,edited,unique_id
0,Human,180,120,mammal,sentient,"brown, blue, green, hazel, grey, amber","blonde, brown, black, red",9.0,Galactic Basic,"caucasian, black, asian, hispanic","[""https://swapi.dev/api/people/66/"", ""https://...","[""https://swapi.dev/api/films/1/"", ""https://sw...",https://swapi.dev/api/species/1/,2014-12-10T13:52:11.567000Z,2014-12-20T21:36:42.136000Z,1


'planets'

,name,diameter,rotation_period,orbital_period,gravity,population,climate,terrain,surface_water,residents,films,url,created,edited,unique_id
0,Tatooine,10465,23,304,1 standard,200000,arid,desert,1,"[""https://swapi.dev/api/people/1/"", ""https://s...","[""https://swapi.dev/api/films/1/"", ""https://sw...",https://swapi.dev/api/planets/1/,2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,1


1. Findings:
    - To join these tables, we need to parse the links in the relevant column.
    - For example, for the table `people`, a single person might appear in several movies, which is an array: `"https://swapi.dev/api/films/1/", "https://sw...`
2. Temporary Solution:
    - Add a column named `unique_id` could be extracted from `url` column.
    - As `homeworld` column only contains a single link, we could get the planet_id in `homework` column to join tables.
3. Solution:
    - For sqlite3, the data type does not support lists, so we could shift to PostgreSQL to do the future data analysis.

In [46]:
# temporary solution to update unique_id and homework column
# tried a sql implementation and found out pandas is more intuitive
def extract_id(url):
    if pd.isnull(url) or not isinstance(url, str):
        return url
    return int(url.strip('/').split('/')[-1])


with sqlite3.connect('star_wars.db') as conn:
    for table in tables:
        df = pd.read_sql(f'select * from {table}', conn)
        
        #create unique_id column for each table
        df['unique_id'] = df['url'].apply(extract_id)

        # update homeworld column from url to id
        if 'homeworld' in df.columns:
            df['homeworld'] = df['homeworld'].apply(extract_id)
        
        df.to_sql(table, conn, if_exists='replace', index=False)

In [50]:
# take a look at the updated tables
for table in tables:
    display(table)
    display(pd.read_sql(f'SELECT * FROM {table} LIMIT 1', conn))

'people'

,name,birth_year,eye_color,gender,hair_color,height,mass,skin_color,homeworld,films,species,starships,vehicles,url,created,edited,unique_id
0,Luke Skywalker,19BBY,blue,male,blond,172,77,fair,1,"[""https://swapi.dev/api/films/1/"", ""https://sw...",[],"[""https://swapi.dev/api/starships/12/"", ""https...","[""https://swapi.dev/api/vehicles/14/"", ""https:...",https://swapi.dev/api/people/1/,2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,1


'films'

,title,episode_id,opening_crawl,director,producer,release_date,species,starships,vehicles,characters,planets,url,created,edited,unique_id
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[""https://swapi.dev/api/species/1/"", ""https://...","[""https://swapi.dev/api/starships/2/"", ""https:...","[""https://swapi.dev/api/vehicles/4/"", ""https:/...","[""https://swapi.dev/api/people/1/"", ""https://s...","[""https://swapi.dev/api/planets/1/"", ""https://...",https://swapi.dev/api/films/1/,2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,1


'starships'

...

,name,model,vehicle_class,manufacturer,length,cost_in_credits,crew,passengers,max_atmosphering_speed,cargo_capacity,consumables,films,pilots,url,created,edited,unique_id
0,Sand Crawler,Digger Crawler,wheeled,Corellia Mining Corporation,36.8,150000,46,30,30,50000,2 months,"[""https://swapi.dev/api/films/1/"", ""https://sw...",[],https://swapi.dev/api/vehicles/4/,2014-12-10T15:36:25.724000Z,2014-12-20T21:30:21.661000Z,4


'species'

,name,average_height,average_lifespan,classification,designation,eye_colors,hair_colors,homeworld,language,skin_colors,people,films,url,created,edited,unique_id
0,Human,180,120,mammal,sentient,"brown, blue, green, hazel, grey, amber","blonde, brown, black, red",9.0,Galactic Basic,"caucasian, black, asian, hispanic","[""https://swapi.dev/api/people/66/"", ""https://...","[""https://swapi.dev/api/films/1/"", ""https://sw...",https://swapi.dev/api/species/1/,2014-12-10T13:52:11.567000Z,2014-12-20T21:36:42.136000Z,1


'planets'

,name,diameter,rotation_period,orbital_period,gravity,population,climate,terrain,surface_water,residents,films,url,created,edited,unique_id
0,Tatooine,10465,23,304,1 standard,200000,arid,desert,1,"[""https://swapi.dev/api/people/1/"", ""https://s...","[""https://swapi.dev/api/films/1/"", ""https://sw...",https://swapi.dev/api/planets/1/,2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,1


## 1. Required SQL analysis

### What's the distribution of planet citizens
- Requirment: `1 request with INNER`

In [55]:
pd.read_sql(
    '''
    select 
        p2.name as planet_name,
        count(p1.name) as people_count
    from people p1
    inner join 
        planets p2
    on p1.homeworld = p2.unique_id
    group by
        p2.name
    order by 
        people_count desc
    ''', 
    conn
)

,planet_name,people_count
0,Naboo,11
1,Tatooine,10
2,unknown,5
3,Kamino,3
4,Coruscant,3
5,Alderaan,3
6,Ryloth,2
7,Mirial,2
8,Kashyyyk,2
9,Corellia,2


In [49]:
pd.read_sql(
    '''
        select 
            vehicle_class,
            group_concat(model) as model,
            round(avg(cost_in_credits),2) as average_cost,
            round(avg(passengers)) as average_passengers,
            round(avg(length), 2) as average_length,
            count(model) as model_count
        from 
            vehicles
        group by
            vehicle_class
        order by 
            average_cost desc
    ''', 
    conn
)

,vehicle_class,model,average_cost,average_passengers,average_length,model_count
0,sail barge,Modified Luxury Sail Barge,285000.00,500.0,30.00,1
1,landing craft,C-9979 landing craft,200000.00,284.0,210.00,1
2,wheeled walker,"Tsmeu-6 personal wheel bike,HAVw A6 Juggernaut",182500.00,151.0,26.45,2
3,wheeled,Digger Crawler,150000.00,30.0,36.80,1
4,droid tank,NR-N99 Persuader-class droid enforcer,49000.00,4.0,10.96,1
5,repulsorcraft,"T-16 skyhopper,X-34 landspeeder,Storm IV Twin-...",35185.71,17.0,9.60,7
6,droid starfighter,tri-fighter,20000.00,0.0,5.40,1
7,air speeder,Raddaugh Gnasp fluttercraft,14750.00,0.0,7.00,1
8,airspeeder,"t-47 airspeeder,Koro-2 Exodrive airspeeder,XJ-...",14425.00,1.0,8.95,5
9,walker,"All Terrain Scout Transport,All Terrain Tactic...",10000.00,17.0,39.60,4


In [ ]:
pd.read_sql(
    '''
        SELECT 
            *
        FROM 
            people
    ''', 
    conn
)